In [15]:
import ee
import geemap
ee.Authenticate()
ee.Initialize()


import pandas as pd
import geopandas as gpd
import numpy as np 
from ipyleaflet import CircleMarker
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from shapely import wkt  # For converting text to geometry

pd.options.display.float_format = '{:.3f}'.format

In [16]:
ref_map = pd.read_csv('/home/sepal-user/COCAFORI/CMR_2020_LC_types_ceo_ref_val.csv')

In [17]:
ref_map_gdf = gpd.GeoDataFrame(ref_map)
# Convert 'geometry' column from string to actual geometry objects
ref_map_gdf['geometry'] = ref_map_gdf['geometry'].apply(wkt.loads)
# Set 'geometry' as the active geometry column
ref_map_gdf = ref_map_gdf.set_geometry('geometry')

In [18]:
ref_map_gdf['lon'] = ref_map_gdf.geometry.x
ref_map_gdf['lat'] = ref_map_gdf.geometry.y

In [19]:
ref_map_gdf.head()

,geometry,LC_2020_Code_Ref,collection,LC_2020_Map,index,plotid,sampleid,lon,lat
0,POINT (13.15682 9.44489),30,1,30,0,2,2,13.157,9.445
1,POINT (14.29857 7.77672),12,1,11,1,4,4,14.299,7.777
2,POINT (14.74055 11.1445),30,1,30,2,7,7,14.741,11.144
3,POINT (13.75959 8.64718),30,1,30,3,6,6,13.760,8.647
4,POINT (13.10112 4.40264),11,1,11,4,8,8,13.101,4.403


In [20]:
# Define conditions and corresponding values
conditions = [
    (ref_map_gdf['LC_2020_Map'].between(10, 19)), 
    (ref_map_gdf['LC_2020_Map'].between(20, 29)),  
    (ref_map_gdf['LC_2020_Map'].between(30, 39)),  
    (ref_map_gdf['LC_2020_Map'].between(40, 49)),  
    (ref_map_gdf['LC_2020_Map'].between(50, 59)),  
    (ref_map_gdf['LC_2020_Map'].between(60, 69)),  
]

values = [10, 20, 30, 40, 50, 60]

# Assign values based on conditions
ref_map_gdf['GIEC_2020_Map'] = np.select(conditions, values, default=np.nan)  # Default to NaN if no condition is met

In [21]:
# Define conditions and corresponding values
conditions = [
    (ref_map_gdf['LC_2020_Code_Ref'].between(10, 19)), 
    (ref_map_gdf['LC_2020_Code_Ref'].between(20, 29)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(30, 39)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(40, 49)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(50, 59)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(60, 69)),  
]

values = [10, 20, 30, 40, 50, 60]

# Assign values based on conditions
ref_map_gdf['GIEC_2020_Ref'] = np.select(conditions, values, default=np.nan)  # Default to NaN if no condition is met

In [22]:
ref_map_gdf['GIEC_agree'] = (ref_map_gdf['GIEC_2020_Map'] == ref_map_gdf['GIEC_2020_Ref']).astype(int)
ref_map_gdf['LC_agree'] = (ref_map_gdf['LC_2020_Map'] == ref_map_gdf['LC_2020_Code_Ref']).astype(int)

In [23]:
ref_map_gdf.head()

,geometry,LC_2020_Code_Ref,collection,LC_2020_Map,index,plotid,sampleid,lon,lat,GIEC_2020_Map,GIEC_2020_Ref,GIEC_agree,LC_agree
0,POINT (13.15682 9.44489),30,1,30,0,2,2,13.157,9.445,30.000,30.000,1,1
1,POINT (14.29857 7.77672),12,1,11,1,4,4,14.299,7.777,10.000,10.000,1,0
2,POINT (14.74055 11.1445),30,1,30,2,7,7,14.741,11.144,30.000,30.000,1,1
3,POINT (13.75959 8.64718),30,1,30,3,6,6,13.760,8.647,30.000,30.000,1,1
4,POINT (13.10112 4.40264),11,1,11,4,8,8,13.101,4.403,10.000,10.000,1,1


In [24]:
ref_map_gdf['GIEC_agree'].value_counts(dropna=False)

GIEC_agree
1    1900
0     517
Name: count, dtype: int64

In [25]:
ref_map_gdf['LC_agree'].value_counts(dropna=False)

LC_agree
1    1539
0     878
Name: count, dtype: int64

In [26]:
ref_map_gdf.to_csv('/home/sepal-user/COCAFORI/CMR_2020_LC_types_ceo_ref_val.csv',index=True)

In [27]:
ref_map_gdf = ref_map_gdf.set_crs("EPSG:4326", inplace=False)

In [28]:
# Initialize Earth Engine
ee.Initialize()

# Create the map
Map = geemap.Map(center=[0, 15], zoom=4)

# Load the GEE landcover image
landcover = ee.Image("projects/ee-cocoacmr/assets/outputs/CMR_forest_types_v6")  # replace with your asset

# Landcover visualization
landcover_vis = {
    'min': 11,
    'max': 60,
    'palette': [
        '#157A46', '#83C16F', '#9E8826', '#9e40b8', '#998056', '#000000', '#000000', '#000000',
        '#000000', '#000000', '#A52A2A', '#AEDF18', '#000000', '#000000', '#000000', '#000000',
        '#000000', '#000000', '#000000', '#C1D3AE', '#000000', '#000000', '#000000', '#000000',
        '#000000', '#000000', '#000000', '#000000', '#000000', '#544C90', '#1B2FEA', '#000000',
        '#000000', '#000000', '#000000', '#000000', '#000000', '#000000', '#784a50', '#000000',
        '#000000', '#000000', '#000000', '#000000', '#000000', '#000000', '#000000', '#000000',
        '#d9d2d0'
    ]
}

# Add landcover to map
Map.addLayer(landcover, landcover_vis, "Landcover")

# --- Load your point layer ---
# Example: ref_map_gdf = gpd.read_file("your_file.geojson")

# Convert LC_agree to colors
# Map LC_agree values to color strings
ref_map_gdf['color'] = ref_map_gdf['LC_agree'].map({0: '#000000', 1: '#544C90'})

Map.add_gdf(ref_map_gdf, layer_name='Ref Points', info_mode='on_hover', style={
    'color': 'color',
    'radius': 5,
    'fillOpacity': 0.8
})


# Show the map
Map


Map(center=[0, 15], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…